# Politic_ES

Creación del modelo

In [41]:
# %matplotlib ipympl

In [96]:
import copy
import os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from spacy.lang.es import Spanish
from time import time
from pandas import DataFrame, Series
from enum import Enum
import nltk

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import IntervalStrategy, EarlyStoppingCallback
from transformers import TextClassificationPipeline
from sklearn.metrics import f1_score, classification_report, precision_recall_fscore_support, accuracy_score

from datasets import Dataset, load_dataset

from tqdm import tqdm

In [3]:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
torch.cuda.empty_cache()

In [4]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
f

0

In [27]:
class ClassifyingTask(Enum):
    def __str__(self):
        return str(self.value)

    IDEO_BIN = 'ideology_binary'
    IDEO_MUL = 'ideology_multiclass'
    GENDER = 'gender'
    PROFESSION = 'profession'


    # # Testing
    # SENTIMENT = 'sentiment'


task = ClassifyingTask.PROFESSION.value

In [6]:
read_df: DataFrame
if ClassifyingTask(task) in [ClassifyingTask.IDEO_BIN, ClassifyingTask.IDEO_MUL, ClassifyingTask.GENDER, ClassifyingTask.PROFESSION]:
    read_df = pd.read_csv(Path('practise_data/politicES_phase_2_train_public.csv'))
    # read_df = read_df.groupby(['label', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass'], as_index=False).agg({'tweet': ''.join})

elif ClassifyingTask(task) in [ClassifyingTask.SENTIMENT]:
    sentiment_path = Path('practise_data/sentiment')

    # Train
    df_train = pd.DataFrame()
    df_train['text'] = pd.read_csv(sentiment_path / 'train_text.txt', sep='\t')
    df_train[task] = pd.read_csv(sentiment_path / 'train_labels.txt', sep='\t')

    # Validation
    df_eval = pd.DataFrame()
    df_eval['text'] = pd.read_csv(sentiment_path / 'val_text.txt', sep='\t')
    df_eval[task] = pd.read_csv(sentiment_path / 'val_labels.txt', sep='\t')

    # Test
    df_test = pd.DataFrame()
    df_test['text'] = pd.read_csv(sentiment_path / 'test_text.txt', sep='\t')
    df_test[task] = pd.read_csv(sentiment_path / 'test_labels.txt', sep='\t')

else:
    raise NotImplementedError

In [7]:
df = read_df.groupby(task).apply(lambda x: x.sample(frac=1))
df

label  gender  profession  \
profession                                                                
celebrity  24315   2206a609406ec61b77ecb9bab9e67879    male   celebrity   
           24004   2184915e947040ca9b6d4fe076d8bcda  female   celebrity   
           51374   4abff7ca77a2fc2ea3ee24f5f1232371    male   celebrity   
           49085   484cb9980c91415a19fb081c7f1633dd    male   celebrity   
           81312   79dd9fa92c8b78a4fcb7c97e1505cfcf    male   celebrity   
...                                             ...     ...         ...   
politician 141455  cb27bb50c60cf653390374aab46c98bb    male  politician   
           159355  e4179a03a9f37ccb28f12644c983fe56    male  politician   
           18754   1bbc976d94bfc296195176281f4dd4c3    male  politician   
           140029  c9857ab0a9604df44deb7f4e205dccb8    male  politician   
           178670  fdf020bc10ce6c9f24474bcf322d4423    male  politician   

                  ideology_binary ideology_multiclass  \
profession                                              
celebrity  24315             left       moderate_left   
           24004            right      moderate_right   
           51374             left       moderate_left   
           49085             left       moderate_left   
           81312            right               right   
...                           ...                 ...   
politician 141455           right      moderate_right   
           159355           right      moderate_right   
           18754             left       moderate_left   
           140029           right               right   
           178670            left                left   

                                                               tweet  
profession                                                            
celebrity  24315   …inestable, inconsecuente, presa d todas sus c...  
           24004   ✌️ Insisto: si @user i @user se hacen docuseri...  
           51374   Gracias, @user  para mí eres muy importante. T...  
           49085   Ya está disponible el nuevo @user donde hablam...  
           81312   @user Él sí q es un peligro para la democracia...  
...                                                              ...  
politician 141455  📲Hoy estaremos en @user analizando la actualid...  
           159355  @user Encantado. ¿Me facilita su teléfono por ...  
           18754   El problema global del medio ambiente puede co...  
           140029  El [POLITICAL_PARTY] se niega a derogar la Ley...  
           178670  @user Que la comunidad internacional tiene dob...  

[180000 rows x 6 columns]

In [8]:
classes_counts = df[task].value_counts()
count_of_smallest = classes_counts.min()
count_of_smallest

9040

In [9]:
df_by_class = [df[df[task] == unique_class] for unique_class in df[task].unique()]

undersampled_df = pd.concat([x.sample(count_of_smallest) for x in df_by_class], axis=0)
undersampled_df[task].value_counts()

celebrity     9040
journalist    9040
politician    9040
Name: profession, dtype: int64

In [10]:
df[task].value_counts()

journalist    110800
politician     60160
celebrity       9040
Name: profession, dtype: int64

In [11]:
frac = 0.1

# df = undersampled_df.sample(frac=frac)  # Equivalent to shuffle
df = df.sample(frac=frac)  # Equivalent to shuffle

In [12]:
model_name: str ="bert-base-multilingual-uncased"  # cardiffnlp/twitter-xlm-roberta-base, xlm-roberta-base, xlm-roberta-large, bert-base-multilingual-uncased, ¿facebook/xlm-roberta-xxl?, ¡cardiffnlp/twitter-xlm-roberta-base-sentiment!
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [13]:
sentence = "How many people did Randy Steven Craft murder?"
print(sentence)
tokenizer(sentence)

How many people did Randy Steven Craft murder?


{'input_ids': [101, 12548, 11260, 11227, 12266, 29841, 17953, 48406, 19428, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
# Set every column as a category
# Set types
for column in ['label', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass']:
    df[column] = df[column].astype ('category')

In [15]:
print(df['label'].value_counts())

4acf2293ccd7815565532700d21914c8    18
6b0cbd8e00c1c6e2c27a7aa6fb123bc9    17
231ed4ce7ed14cb38055426acd5d9747    17
b21d6bcb39422349d9f3ceff3a49d97a    17
e11fb93bf6feec71d81a8caf1bd36b5f    17
                                    ..
ca328738a9bde9b06d68784e76c98595     2
cfc8beefbc8774e16275fb8e0c5775b4     2
535b7ac18b93116611762b9c267f1916     1
7afa97d01115a682e7a1793302455380     1
aca31cd35236af4d561c70bbbd60eada     1
Name: label, Length: 2250, dtype: int64


Limpieza de los textos (principalmente eliminación de emojis, pero también la simplificación de los números)

In [134]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

spanish_tokenizer = Spanish()

# nltk.download('stopwords')
# spanish_stopwords = nltk.corpus.stopwords.words('spanish')

def spanish(data):
    return ''.join([token.text_with_ws for token in spanish_tokenizer(data)])

prev_column_names = ['label', 'tweet', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass']


df_clean = df[prev_column_names].rename(columns={'tweet': 'text', 'label': 'original_label'})
df_clean['text'] = df_clean['text'].apply(remove_emojis).apply(spanish)
df_clean['num_chars'] = df_clean.apply(lambda row: len(row['text']), axis=1)
df_clean['num_tokens'] = df_clean.apply(lambda row: len(tokenizer(row['text'])['input_ids']), axis=1)

basic_column_names = ['original_label', 'text', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass', 'num_chars', 'num_tokens']

df_clean

original_label  \
profession                                            
journalist 140140  c9b4c82ac42ba597af950044bac8801f   
politician 104560  99f6cb905f850b1a944f11a83b56f3b1   
journalist 77105   732f91c944ec6f848fe19944fef4ba23   
politician 39384   3b455b15e6787b61ac69a684bdce92a1   
journalist 160718  e5f614ab17fa138a4171e88a42e62c40   
...                                             ...   
politician 132074  bfa38c347bf6bb98a76f78d57b0c402d   
           172425  f5aa855b9c05dfd62826c90fe955a3b0   
journalist 15654   16f6536ac4e4b2169282b6e25fca24c6   
           79177   764596a8afa8f17e2f40885a0561a684   
           151957  d98dc332cc875450a2c19963ec894776   

                                                                text  gender  \
profession                                                                     
journalist 140140  El comité que ha investigado el asalto del Cap...    male   
politician 104560  ¿El Emérito quiere ocultar su delitos con su t...    male   
journalist 77105   Me han traído un router, ha venido un técnico ...    male   
politician 39384   Asturias mantendrá el abono transporte a 30 eu...    male   
journalist 160718  @user También me pareció absurdo eso, sí. Otra...    male   
...                                                              ...     ...   
politician 132074  No entiendo por qué se llama la etapa de [HASH...  female   
           172425  En la plaza, con la muestra 'Palabra de Cádiz'...    male   
journalist 15654   @user O canónigos como el de Cajasur, un sinve...    male   
           79177   ¿Sabéis el remedio cuando nunca aceptan tus di...    male   
           151957  Ojo al último informe de la Guardia Civil sobr...  female   

                   profession ideology_binary ideology_multiclass  num_chars  \
profession                                                                     
journalist 140140  journalist            left       moderate_left        208   
politician 104560  politician            left                left        279   
journalist 77105   journalist            left       moderate_left        269   
politician 39384   politician            left       moderate_left         78   
journalist 160718  journalist           right      moderate_right        195   
...                       ...             ...                 ...        ...   
politician 132074  politician            left       moderate_left        145   
           172425  politician            left                left        265   
journalist 15654   journalist            left       moderate_left         54   
           79177   journalist           right               right         75   
           151957  journalist            left       moderate_left        210   

                   num_tokens  
profession                     
journalist 140140          43  
politician 104560          76  
journalist 77105           71  
politician 39384           19  
journalist 160718          53  
...                       ...  
politician 132074          42  
           172425          73  
journalist 15654           20  
           79177           24  
           151957          46  

[18000 rows x 8 columns]

In [135]:
MAX_LENGTH = df_clean['num_tokens'].max()
MAX_LENGTH

129

# Separación en train y test

In [136]:
from typing import List


def get_tokenizer_extra_columns(model_name: str) -> List[str]:
    return ['input_ids', 'token_type_ids', 'attention_mask'] if not "xlm-roberta" in model_name else ['input_ids', 'attention_mask']

In [142]:
# Separamos en training y en test
parte_eval = 0.15
parte_test = 0.15

len_df = len(df)
punto_eval = int(parte_eval * len_df)
punto_test = int((parte_eval + parte_test) * len_df)

df_train: DataFrame
df_eval: DataFrame
df_test: DataFrame
df_eval, df_test, df_train = df_clean.iloc[:punto_eval], df_clean.iloc[punto_eval:punto_test], df_clean.iloc[punto_test:]

In [143]:
def tokker(x):
    if "xlm-roberta" in model_name:
        res = tokenizer(x['text'], padding="max_length", truncation=True, max_length=MAX_LENGTH)
        input_ids, attention_mask = res['input_ids'], res['attention_mask']
        return Series(data=(*[col for col in x], input_ids, attention_mask), index = basic_column_names + get_tokenizer_extra_columns(model_name))
    else:
        res = tokenizer(x['text'], padding="max_length", truncation=True, max_length=MAX_LENGTH)
        input_ids, token_type_ids, attention_mask = res['input_ids'], res['token_type_ids'], res['attention_mask']
        return Series(data=(*[col for col in x], input_ids, token_type_ids, attention_mask), index = basic_column_names + get_tokenizer_extra_columns(model_name))

In [144]:
df_train = df_train.apply(tokker, axis=1, result_type='expand')
df_train

original_label  \
profession                                            
journalist 99843   92a3f751e8d866f816a0b2e73c56ebdc   
politician 115242  a81281e5aeaceb467650a638f4581246   
journalist 107376  9d2e209ca895abdac0ef213642e8979b   
           43350   404d9c7842520b032f85133bb35a7662   
           130046  bc9e7a55ce2669390f12e7dbaac52410   
...                                             ...   
politician 132074  bfa38c347bf6bb98a76f78d57b0c402d   
           172425  f5aa855b9c05dfd62826c90fe955a3b0   
journalist 15654   16f6536ac4e4b2169282b6e25fca24c6   
           79177   764596a8afa8f17e2f40885a0561a684   
           151957  d98dc332cc875450a2c19963ec894776   

                                                                text  gender  \
profession                                                                     
journalist 99843   @user ¿Estamos seguros de que es por acumulaci...  female   
politician 115242  Buenas noches a todos menos a los que suelen e...  female   
journalist 107376  @user No es cuestión solo de festivales, por m...    male   
           43350   La viñeta de hoy de @user pronto será realidad...    male   
           130046  ¡Cien corridas! ¡Enhorabuena y gracias por tod...    male   
...                                                              ...     ...   
politician 132074  No entiendo por qué se llama la etapa de [HASH...  female   
           172425  En la plaza, con la muestra 'Palabra de Cádiz'...    male   
journalist 15654   @user O canónigos como el de Cajasur, un sinve...    male   
           79177   ¿Sabéis el remedio cuando nunca aceptan tus di...    male   
           151957  Ojo al último informe de la Guardia Civil sobr...  female   

                   profession ideology_binary ideology_multiclass  num_chars  \
profession                                                                     
journalist 99843   journalist            left       moderate_left        211   
politician 115242  politician           right               right         87   
journalist 107376  journalist            left       moderate_left        205   
           43350   journalist           right      moderate_right        167   
           130046  journalist           right               right        183   
...                       ...             ...                 ...        ...   
politician 132074  politician            left       moderate_left        145   
           172425  politician            left                left        265   
journalist 15654   journalist            left       moderate_left         54   
           79177   journalist           right               right         75   
           151957  journalist            left       moderate_left        210   

                   num_tokens  \
profession                      
journalist 99843           56   
politician 115242          27   
journalist 107376          60   
           43350           47   
           130046          49   
...                       ...   
politician 132074          42   
           172425          73   
journalist 15654           20   
           79177           24   
           151957          46   

                                                           input_ids  \
profession                                                             
journalist 99843   [101, 137, 24934, 200, 10602, 14000, 65099, 10...   
politician 115242  [101, 84542, 25605, 10107, 143, 12556, 14594, ...   
journalist 107376  [101, 137, 24934, 10181, 10153, 61977, 11174, ...   
           43350   [101, 10106, 63352, 10211, 10102, 21150, 10102...   
           130046  [101, 173, 66901, 64949, 10107, 106, 173, 1010...   
...                                                              ...   
politician 132074  [101, 10181, 28960, 19676, 10190, 10126, 10128...   
           172425  [101, 10109, 10106, 15133, 117, 10173, 10106, ...   
journalist 15654   [101, 137, 24934, 157, 20585, 30186, 10107, 10...   
           79177 

In [23]:
df_eval = df_eval.apply(tokker, axis=1, result_type='expand')
df_eval

label  \
profession                                            
journalist 140140  c9b4c82ac42ba597af950044bac8801f   
politician 104560  99f6cb905f850b1a944f11a83b56f3b1   
journalist 77105   732f91c944ec6f848fe19944fef4ba23   
politician 39384   3b455b15e6787b61ac69a684bdce92a1   
journalist 160718  e5f614ab17fa138a4171e88a42e62c40   
...                                             ...   
           130140  bcbf5d6ff4082d747a9f95eab9877572   
           146954  d2fd365758464bf40b1312dbdcc8508e   
politician 30215   2c5fdbb3582ee8294fed0e7dbd0d52bc   
journalist 42874   3fece7d406218f6f6042b9e7b2bfd47c   
politician 58502   542cbbe7f1ad6723d517dd92da4e38de   

                                                                text  gender  \
profession                                                                     
journalist 140140  El comité que ha investigado el asalto del Cap...    male   
politician 104560  ¿El Emérito quiere ocultar su delitos con su t...    male   
journalist 77105   Me han traído un router, ha venido un técnico ...    male   
politician 39384   Asturias mantendrá el abono transporte a 30 eu...    male   
journalist 160718  @user También me pareció absurdo eso, sí. Otra...    male   
...                                                              ...     ...   
           130140  @user La mejor selección de música en el mejor...  female   
           146954  @user Años y años de pérdidas habrán merecido ...    male   
politician 30215   @user La giga factoría y el centro de investig...    male   
journalist 42874   El BarMar será el mayor hidroducto del mundo, ...    male   
politician 58502   No imagino mejor lugar para celebrar el día de...  female   

                   profession ideology_binary ideology_multiclass  num_chars  \
profession                                                                     
journalist 140140  journalist            left       moderate_left        208   
politician 104560  politician            left                left        279   
journalist 77105   journalist            left       moderate_left        269   
politician 39384   politician            left       moderate_left         78   
journalist 160718  journalist           right      moderate_right        195   
...                       ...             ...                 ...        ...   
           130140  journalist           right      moderate_right        125   
           146954  journalist           right      moderate_right         54   
politician 30215   politician            left       moderate_left         87   
journalist 42874   journalist            left       moderate_left        116   
politician 58502   politician           right      moderate_right        256   

                   num_tokens  \
profession                      
journalist 140140          43   
politician 104560          76   
journalist 77105           71   
politician 39384           19   
journalist 160718          53   
...                       ...   
           130140          32   
           146954          17   
politician 30215           23   
journalist 42874           32   
politician 58502           59   

                                                           input_ids  \
profession                                                             
journalist 140140  [101, 10117, 17532, 10126, 10240, 10104, 54093...   
politician 104560  [101, 200, 10117, 84017, 40720, 63644, 29707, ...   
journalist 77105   [101, 10525, 10210, 18352, 10351, 10119, 12724...   
politician 39384   [101, 39830, 10564, 67792, 10281, 10117, 59520...   
journalist 160718  [101, 137, 24934, 11382, 10525, 16226, 10973, ...   
...                                                              ...   
           130140  [101, 137, 24934, 10106, 16440, 22695, 10102, ...   
           146954  [101, 137, 24934, 10850, 167, 10850, 10102, 42...   
politician 30215   [101, 137, 24934, 10106, 21464, 10547, 18482, ...   
journalist 42874   [101, 1

In [24]:
df_test = df_test.apply(tokker, axis=1, result_type='expand')
df_test

label  \
profession                                            
journalist 85977   7fd2b09b530796e8f19a0e7ed847573b   
           45264   4358b83e373b0bb90b4a2b60060363b1   
politician 1213    01893a6c2c0b2787aaa382bb65b88115   
journalist 161055  e66998b63cda1ebf0a6805ec45767adf   
           125215  b54c002c3ca54a86e507a11ad2abb5a3   
...                                             ...   
politician 53088   4cedad4067c3cbdd180e27d8eb241cc8   
journalist 434     007d091d5f6b562acdba68ddc5d0fdb8   
           128075  ba1175b372f83805347a8cd69aa3b11d   
           2378    026d6a1eaa4f3378eaa67ca455b22ef3   
politician 140245  c9c56976299ed4a3860a88c4d2d00aae   

                                                                text  gender  \
profession                                                                     
journalist 85977   Ya puedes escuchar: Programa Completo de La Vo...    male   
           45264   Una juez suspende la emisión del sello que con...    male   
politician 1213    Como bien ha explicado el Ministro @user en la...  female   
journalist 161055  'Big Four': conflictos de interés y captura de...    male   
           125215  ¿Quién es el mejor futbolista de todos los tie...    male   
...                                                              ...     ...   
politician 53088   En la ponencia que regula la PAC junto a @user...    male   
journalist 434     Hubo que soportar en pandemia a quienes decían...  female   
           128075  Lo más parecido a lo que propone [POLITICIAN] ...    male   
           2378    A punto de comenzar la segunda sesión del Semi...  female   
politician 140245  Repugnante, miserable es como el representante...  female   

                   profession ideology_binary ideology_multiclass  num_chars  \
profession                                                                     
journalist 85977   journalist           right      moderate_right         73   
           45264   journalist            left                left        199   
politician 1213    politician            left       moderate_left        227   
journalist 161055  journalist            left       moderate_left         79   
           125215  journalist           right      moderate_right         71   
...                       ...             ...                 ...        ...   
politician 53088   politician           right               right        200   
journalist 434     journalist           right      moderate_right        267   
           128075  journalist            left       moderate_left        256   
           2378    journalist            left       moderate_left        156   
politician 140245  politician            left       moderate_left        283   

                   num_tokens  \
profession                      
journalist 85977           23   
           45264           43   
politician 1213            54   
journalist 161055          22   
           125215          19   
...                       ...   
politician 53088           42   
journalist 434             71   
           128075          64   
           2378            34   
politician 140245          75   

                                                           input_ids  \
profession                                                             
journalist 85977   [101, 10593, 13139, 10107, 10153, 11575, 44342...   
           45264   [101, 10155, 78442, 10877, 95111, 10106, 75516...   
politician 1213    [101, 10245, 11338, 10240, 40056, 10351, 10117...   
journalist 161055  [101, 112, 12062, 11443, 112, 131, 80149, 1010...   
           125215  [101, 200, 13829, 10153, 10117, 16440, 30158, ...   
...                                                              ...   
politician 53088   [101, 10109, 10106, 53138, 13823, 10126, 95364...   
journalist 434     [101, 35553, 10126, 10297, 73500, 10131, 10109...   
           128075  [101, 10387, 10321, 95133, 143, 10387, 10126, ...   
           2378    [101, 1

### Guardar como Dataset

In [28]:
# Labels se cambian a números para el Transformer
gender2num = {
    'male': 0,
    'female': 1,
}

profession2num = {
    'journalist': 0,
    'celebrity': 1,
    'politician': 2,
}

ideology_binary2num = {
    'left': 0,
    'right': 1,
}

ideology_multiclass2num = {
    'left': 0,
    'moderate_left': 1,
    'moderate_right': 2,
    'right': 3,
}

sentiment2num = {
    0: 0,  # Negative
    1: 1,  # Neutral
    2: 2,  # Positive
}


dict_for_conversion = None

if ClassifyingTask(task) is ClassifyingTask.IDEO_BIN:
    dict_for_conversion = ideology_binary2num
elif ClassifyingTask(task) is ClassifyingTask.IDEO_MUL:
    dict_for_conversion = ideology_multiclass2num
elif ClassifyingTask(task) is ClassifyingTask.GENDER:
    dict_for_conversion = gender2num
elif ClassifyingTask(task) is ClassifyingTask.PROFESSION:
    dict_for_conversion = profession2num
else:
    raise NotImplementedError

inv_dict_for_conversion = {v: k for k, v in dict_for_conversion.items()}

df_train_numeric_labels = df_train.copy()
df_eval_numeric_labels = df_eval.copy()
df_test_numeric_labels = df_test.copy()

df_train_numeric_labels[task] = df_train_numeric_labels[task].apply(lambda x: dict_for_conversion[x])
df_eval_numeric_labels[task] = df_eval_numeric_labels[task].apply(lambda x: dict_for_conversion[x])
df_test_numeric_labels[task] = df_test_numeric_labels[task].apply(lambda x: dict_for_conversion[x])

dataset_train = Dataset.from_pandas(df_train_numeric_labels).rename_column(task, 'labels').rename_column('label', 'original_label')
dataset_eval = Dataset.from_pandas(df_eval_numeric_labels).rename_column(task, 'labels').rename_column('label', 'original_label')
dataset_test = Dataset.from_pandas(df_test_numeric_labels).rename_column(task, 'labels').rename_column('label', 'original_label')

## Modelo

In [89]:
import gc
try:
    del model
    torch.cuda.empty_cache()
    gc.collect()
    print("Cleared")
except:
    pass


Cleared


In [90]:
TARGET_LABELS = [label for label in dict_for_conversion]
NUM_LABELS = len(TARGET_LABELS)

print('TARGET_LABELS:', TARGET_LABELS, 'num_labels:', NUM_LABELS)

# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)
model = AutoModelForSequenceClassification.from_pretrained(f'./models/best_model_so_far/{model_name}/{task}/', local_files_only=True, num_labels=NUM_LABELS)

TARGET_LABELS: ['journalist', 'celebrity', 'politician'] num_labels: 3


In [91]:
args = TrainingArguments(output_dir="./outputs")

In [92]:
# args.remove_unused_columns = False
args.save_strategy = 'no'

args.evaluation_strategy="epoch"
args.num_train_epochs = 10

# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = 10
# args.max_steps = 100


args.learning_rate = 5E-6

args.load_best_model_at_end = True  # EarlyStoppingCallback() requires this to be True
args.metric_for_best_model = 'f1'

# args.per_device_train_batch_size = 8
# args.per_device_eval_batch_size = 8
# args.auto_find_batch_size = True


In [93]:
def compute_metrics(pred):
    """recibe un lote predicciones inferidas por el modelo. """

    y_true = pred.label_ids # son las labels en el gold standard
    y_pred = pred.predictions.argmax(-1)    # pred.predictions devuelve una lista con las predicciones
                                            # para cada clase. Debemos quedarnos con la de mayor probabilidad.

    # como son varias clases, utilizaremos la macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
    acc = accuracy_score(y_true, y_pred)

    to_return = {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    print(to_return)

    return to_return

In [94]:
trainer = Trainer(
    model=model,            # modelo que será ajustado
    args=args,     # hiperparámetros
    train_dataset=dataset_train, # conjunto training
    eval_dataset=dataset_eval,   # conjunto de validación
    compute_metrics=compute_metrics,    # función para computar las métricas
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [95]:
tokenizer.model_max_length

512

In [211]:
# trainer.train()

C:\Users\1bran\Anaconda3\envs\PLN_PoliticES\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


{'accuracy': 0.7440740740740741, 'f1': 0.47984152280540826, 'precision': 0.5006542608384374, 'recall': 0.48180982441438064}
{'accuracy': 0.7488888888888889, 'f1': 0.5254561416345145, 'precision': 0.7201633607983862, 'recall': 0.5228850194772098}
{'accuracy': 0.7570370370370371, 'f1': 0.5351191827178713, 'precision': 0.6974908231714408, 'recall': 0.5250513495278971}
{'accuracy': 0.7477777777777778, 'f1': 0.5461400507184337, 'precision': 0.6197767250162068, 'recall': 0.5391301162811927}
{'accuracy': 0.7311111111111112, 'f1': 0.5530041315431793, 'precision': 0.5962143902549928, 'recall': 0.5489297085097834}
{'accuracy': 0.7137037037037037, 'f1': 0.532717748627666, 'precision': 0.5732088182981995, 'recall': 0.5396950303816032}
{'accuracy': 0.7381481481481481, 'f1': 0.5442350246282639, 'precision': 0.5876100769412375, 'recall': 0.5385323208883938}
{'accuracy': 0.7296296296296296, 'f1': 0.5510092153803297, 'precision': 0.5894028028356386, 'recall': 0.5468263903915043}
{'accuracy': 0.73148148

TrainOutput(global_step=15750, training_loss=0.37982564629448784, metrics={'train_runtime': 2681.4968, 'train_samples_per_second': 46.989, 'train_steps_per_second': 5.874, 'total_flos': 9324081740736000.0, 'train_loss': 0.37982564629448784, 'epoch': 10.0})

In [86]:
# trainer.save_model(output_dir=Path(f"./models/best_model_so_far/{model_name}/{task}/"))

In [96]:
trainer.evaluate(dataset_test)

{'accuracy': 0.7203703703703703, 'f1': 0.5461534972225347, 'precision': 0.5669095044440805, 'recall': 0.5410099547141297}


{'eval_loss': 1.7037737369537354,
 'eval_accuracy': 0.7203703703703703,
 'eval_f1': 0.5461534972225347,
 'eval_precision': 0.5669095044440805,
 'eval_recall': 0.5410099547141297,
 'eval_runtime': 15.4237,
 'eval_samples_per_second': 175.056,
 'eval_steps_per_second': 21.914}

# Predicción

Conglomeramos todos los modelos en un diccionario para hacer las predicciones sobre un archivo


In [29]:
all_models = {}

for task in ClassifyingTask:
    if ClassifyingTask(task) is ClassifyingTask.IDEO_BIN:
        dict_for_conversion = ideology_binary2num
    elif ClassifyingTask(task) is ClassifyingTask.IDEO_MUL:
        dict_for_conversion = ideology_multiclass2num
    elif ClassifyingTask(task) is ClassifyingTask.GENDER:
        dict_for_conversion = gender2num
    elif ClassifyingTask(task) is ClassifyingTask.PROFESSION:
        dict_for_conversion = profession2num
    else:
        print(task)
        raise NotImplementedError

    TARGET_LABELS = [label for label in dict_for_conversion]
    NUM_LABELS = len(TARGET_LABELS)

    model_for_task = AutoModelForSequenceClassification.from_pretrained(Path(f'./models/best_model_so_far/{model_name}/{task}/'), local_files_only=True, num_labels=NUM_LABELS)

    # Save into dictionary
    all_models[task.value] = {
        'predict': TextClassificationPipeline(model=model_for_task, tokenizer=tokenizer, top_k=None),
        'test_wrapper': Trainer(model=model_for_task, train_dataset=dataset_train, eval_dataset=dataset_eval)  # Not to be used for training, only testing
    }



Antes de seguir, hacemos que los datasets se ajusten a la tarea sobre la que se quiere hacer predicciones.

In [30]:
task = ClassifyingTask.IDEO_MUL.value

# Labels se cambian a números para el Transformer
gender2num = {
    'male': 0,
    'female': 1,
}

profession2num = {
    'journalist': 0,
    'celebrity': 1,
    'politician': 2,
}

ideology_binary2num = {
    'left': 0,
    'right': 1,
}

ideology_multiclass2num = {
    'left': 0,
    'moderate_left': 1,
    'moderate_right': 2,
    'right': 3,
}

sentiment2num = {
    0: 0,  # Negative
    1: 1,  # Neutral
    2: 2,  # Positive
}


dict_for_conversion = None

if ClassifyingTask(task) is ClassifyingTask.IDEO_BIN:
    dict_for_conversion = ideology_binary2num
elif ClassifyingTask(task) is ClassifyingTask.IDEO_MUL:
    dict_for_conversion = ideology_multiclass2num
elif ClassifyingTask(task) is ClassifyingTask.GENDER:
    dict_for_conversion = gender2num
elif ClassifyingTask(task) is ClassifyingTask.PROFESSION:
    dict_for_conversion = profession2num
elif ClassifyingTask(task) is ClassifyingTask.SENTIMENT:
    dict_for_conversion = sentiment2num
else:
    raise NotImplementedError

inv_dict_for_conversion = {v: k for k, v in dict_for_conversion.items()}

df_train_numeric_labels = df_train.copy()
df_eval_numeric_labels = df_eval.copy()
df_test_numeric_labels = df_test.copy()

df_train_numeric_labels[task] = df_train_numeric_labels[task].apply(lambda x: dict_for_conversion[x])
df_eval_numeric_labels[task] = df_eval_numeric_labels[task].apply(lambda x: dict_for_conversion[x])
df_test_numeric_labels[task] = df_test_numeric_labels[task].apply(lambda x: dict_for_conversion[x])

dataset_train = Dataset.from_pandas(df_train_numeric_labels).rename_column(task, 'labels').rename_column('label', 'original_label')
dataset_eval = Dataset.from_pandas(df_eval_numeric_labels).rename_column(task, 'labels').rename_column('label', 'original_label')
dataset_test = Dataset.from_pandas(df_test_numeric_labels).rename_column(task, 'labels').rename_column('label', 'original_label')

In [31]:
dataset_test.features

{'original_label': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'profession': Value(dtype='string', id=None),
 'ideology_binary': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'num_chars': Value(dtype='int64', id=None),
 'num_tokens': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='string', id=None),
 '__index_level_1__': Value(dtype='int64', id=None)}

In [162]:
output_df = pd.read_csv(Path(f'./practise_data/politicES_phase_2_train_public.csv')).sample(frac=0.1)

output_df_clean = output_df[['label', 'tweet', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass']].rename(columns={'tweet': 'text', 'label': 'original_label'})
output_df_clean['text'] = output_df_clean['text'].apply(remove_emojis).apply(spanish)
output_df_clean['num_chars'] = output_df_clean.apply(lambda row: len(row['text']), axis=1)
output_df_clean['num_tokens'] = output_df_clean.apply(lambda row: len(tokenizer(row['text'])['input_ids']), axis=1)

output_df_clean = output_df_clean.apply(tokker, axis=1, result_type='expand')
output_df_clean

,original_label,text,gender,profession,ideology_binary,ideology_multiclass,num_chars,num_tokens,input_ids,token_type_ids,attention_mask
15706,16f89cf61eb0d466c42f9f1f8c82cf55,Enhorabuena @user y toda la fuerza del mundo p...,male,politician,right,right,115,33,"[101, 10109, 56152, 64490, 10206, 137, 24934, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
87922,8408dae0e7f64fc71a02170afa4a4758,Esta tarde acompañando al Círculo Taurino de B...,male,politician,right,moderate_right,204,49,"[101, 10602, 14275, 12181, 10785, 48101, 10622...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
160893,e6273df009c7fbdad831ea8b49276b4b,Este miércoles tocamos en Madrid.. Tengo muchí...,male,celebrity,left,moderate_left,61,22,"[101, 10494, 95096, 66849, 11300, 62017, 14000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
82621,7b2fd78bf2f7667d063f7db9f0089006,"Hoy en [HASHTAG] , ‘Argentina, 1985’, el juici...",male,journalist,left,moderate_left,196,61,"[101, 21150, 10109, 138, 10438, 31340, 10251, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
58622,5438bb857031e2402a4ec4ae918f0ce4,Susto en avión de Aeroméxico: por captura de O...,male,journalist,right,moderate_right,84,24,"[101, 10877, 10366, 10109, 25361, 10102, 48753...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...
113826,a6617201a3316d4eb060e695afd15b8f,Tremendo el reportaje de @user sobre las resid...,male,journalist,left,moderate_left,84,25,"[101, 11555, 11719, 10351, 10117, 13100, 20515...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
87957,8408dae0e7f64fc71a02170afa4a4758,@user Ya tiene trabajo el comité de bulos que ...,male,politician,right,moderate_right,122,34,"[101, 137, 24934, 10593, 12687, 15858, 10117, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
134975,c2bbb321be752f0dcc1ddac450bb69e2,"Hemos mantenido en Crawley (Reino Unido), una ...",male,politician,left,moderate_left,270,61,"[101, 14308, 10327, 10564, 60795, 10109, 24590...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
136288,c5651b5c96904a49788fe60fb41e956a,La degradación de nuestra clase política es ta...,male,journalist,right,moderate_right,280,62,"[101, 10106, 88831, 18179, 10985, 10102, 30571...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [163]:
from datasets import disable_caching
disable_caching()

# Archivo sobre el que hacer salida
output_dataset = Dataset.from_pandas(output_df_clean)
output_dataset

Dataset({
    features: ['original_label', 'text', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass', 'num_chars', 'num_tokens', 'input_ids', 'token_type_ids', 'attention_mask', '__index_level_0__'],
    num_rows: 18000
})

In [164]:
output_dataset['profession']

['politician',
 'politician',
 'celebrity',
 'journalist',
 'journalist',
 'journalist',
 'politician',
 'journalist',
 'journalist',
 'journalist',
 'journalist',
 'journalist',
 'politician',
 'politician',
 'journalist',
 'politician',
 'journalist',
 'politician',
 'journalist',
 'journalist',
 'journalist',
 'politician',
 'celebrity',
 'politician',
 'politician',
 'celebrity',
 'politician',
 'politician',
 'journalist',
 'journalist',
 'journalist',
 'politician',
 'journalist',
 'politician',
 'politician',
 'journalist',
 'politician',
 'journalist',
 'politician',
 'journalist',
 'journalist',
 'politician',
 'politician',
 'politician',
 'journalist',
 'journalist',
 'journalist',
 'journalist',
 'politician',
 'politician',
 'politician',
 'journalist',
 'journalist',
 'politician',
 'journalist',
 'politician',
 'journalist',
 'politician',
 'journalist',
 'journalist',
 'politician',
 'politician',
 'celebrity',
 'politician',
 'politician',
 'politician',
 'politician',

In [165]:
def to_numeric(x: dict) -> dict:
    original_label = x[str(t.value)]
    numerical_label = dict_for_conversion[original_label]
    x[str(t.value)] = numerical_label
    return x

for t in ClassifyingTask:
    dict_for_conversion = None
    if ClassifyingTask(t) is ClassifyingTask.IDEO_BIN:
        dict_for_conversion = ideology_binary2num
    elif ClassifyingTask(t) is ClassifyingTask.IDEO_MUL:
        dict_for_conversion = ideology_multiclass2num
    elif ClassifyingTask(t) is ClassifyingTask.GENDER:
        dict_for_conversion = gender2num
    elif ClassifyingTask(t) is ClassifyingTask.PROFESSION:
        dict_for_conversion = profession2num
    else:
        raise NotImplementedError

    output_dataset = output_dataset.map(to_numeric, load_from_cache_file=False)

C:\Users\1bran\Anaconda3\envs\PLN_PoliticES\lib\site-packages\dill\_dill.py:1705: PicklingWarning: Cannot locate reference to <enum 'ClassifyingTask'>.
  warnings.warn('Cannot locate reference to %r.' % (obj,), PicklingWarning)
C:\Users\1bran\Anaconda3\envs\PLN_PoliticES\lib\site-packages\dill\_dill.py:1707: PicklingWarning: Cannot pickle <enum 'ClassifyingTask'>: __main__.ClassifyingTask has recursive self-references that trigger a RecursionError.
  warnings.warn('Cannot pickle %r: %s.%s has recursive self-references that trigger a RecursionError.' % (obj, obj.__module__, obj_name), PicklingWarning)


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

In [184]:
c_reps = []  # Classification reports

# For each task, we change the column name to 'labels', make the predictions, get the mode and have that be the output
for t in ClassifyingTask:
    print(t)

    output_dataset_for_t = copy.deepcopy(output_dataset).rename_column(t.value, 'labels')
    pred_output = all_models[task]['test_wrapper'].predict(output_dataset_for_t)

    # Into a DF
    comparison_df = pd.DataFrame()
    comparison_df['original_label'] = output_dataset_for_t['original_label']
    comparison_df['text'] = output_dataset_for_t['text']
    comparison_df['labels'] = pred_output.label_ids
    comparison_df['predictions'] = pred_output.predictions.argmax(-1)

    # Grouped by mode
    grouping = comparison_df.groupby('original_label')
    mode_comparison_df = grouping.agg(lambda x: pd.Series.mode(x)[0])
    # mode_comparison_df['considered_predictions'] = grouping.agg(pd.Series.mode)['predictions']
    mode_comparison_df['count'] = grouping.size()

    # Print results
    cr_labels = list(mode_comparison_df['labels'].unique())

    dict_for_conversion = None

    if ClassifyingTask(t) is ClassifyingTask.GENDER:
        dict_for_conversion = gender2num
    elif ClassifyingTask(t) is ClassifyingTask.IDEO_BIN:
        dict_for_conversion = ideology_binary2num
    elif ClassifyingTask(t) is ClassifyingTask.IDEO_MUL:
        dict_for_conversion = ideology_multiclass2num
    elif ClassifyingTask(t) is ClassifyingTask.PROFESSION:
        dict_for_conversion = profession2num
    else:
        raise NotImplementedError

    print(classification_report(mode_comparison_df['labels'], mode_comparison_df['predictions'], labels=[target_label for target_label in dict_for_conversion.values()], target_names=[target_name for target_name in dict_for_conversion], zero_division=0))

    # Save results
    c_reps.append(classification_report(mode_comparison_df['labels'], mode_comparison_df['predictions'], zero_division=0, output_dict=True))


ideology_binary
              precision    recall  f1-score   support

        left       0.71      0.34      0.46      1255
       right       0.25      0.17      0.21       995

   micro avg       0.46      0.27      0.34      2250
   macro avg       0.48      0.26      0.33      2250
weighted avg       0.50      0.27      0.35      2250

ideology_multiclass


                precision    recall  f1-score   support

          left       0.40      0.56      0.47       430
 moderate_left       0.63      0.53      0.58       825
moderate_right       0.59      0.51      0.55       728
         right       0.51      0.60      0.55       267

      accuracy                           0.54      2250
     macro avg       0.53      0.55      0.54      2250
  weighted avg       0.56      0.54      0.54      2250

gender


              precision    recall  f1-score   support

        male       0.62      0.25      0.36      1493
      female       0.41      0.38      0.40       757

   micro avg       0.51      0.29      0.37      2250
   macro avg       0.52      0.32      0.38      2250
weighted avg       0.55      0.29      0.37      2250

profession


              precision    recall  f1-score   support

  journalist       0.60      0.26      0.36      1385
   celebrity       0.07      0.44      0.12       113
  politician       0.32      0.27      0.30       752

   micro avg       0.32      0.27      0.29      2250
   macro avg       0.33      0.33      0.26      2250
weighted avg       0.48      0.27      0.33      2250



In [183]:
print(classification_report(mode_comparison_df['labels'], mode_comparison_df['predictions'], zero_division=0))

              precision    recall  f1-score   support

           0       0.71      0.34      0.46      1255
           1       0.25      0.17      0.21       995
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

    accuracy                           0.27      2250
   macro avg       0.24      0.13      0.17      2250
weighted avg       0.50      0.27      0.35      2250



# Producción de archivo de salida

Aquí se produce el archivo de salida con todas las predicciones

In [38]:
# Archivo sobre el que hacer salida
output_dataset = load_dataset('csv', data_files=str(Path(f'./practise_data/politicES_phase_2_train_public.csv')))

output_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/1bran/.cache/huggingface/datasets/csv/default-e98f946569ec0acc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass', 'tweet'],
        num_rows: 180000
    })
})

In [40]:
for t in all_models:
    pred_output_t = all_models[t]['test_wrapper'].predict(output_dataset)


{'predict': <transformers.pipelines.text_classification.TextClassificationPipeline object at 0x0000026984540DF0>, 'test_wrapper': <transformers.trainer.Trainer object at 0x00000268EEF813D0>}
{'predict': <transformers.pipelines.text_classification.TextClassificationPipeline object at 0x00000269C90CBFA0>, 'test_wrapper': <transformers.trainer.Trainer object at 0x00000269C90CBE20>}
{'predict': <transformers.pipelines.text_classification.TextClassificationPipeline object at 0x00000269C90B3640>, 'test_wrapper': <transformers.trainer.Trainer object at 0x00000269C90B32E0>}
{'predict': <transformers.pipelines.text_classification.TextClassificationPipeline object at 0x0000026985BBCCD0>, 'test_wrapper': <transformers.trainer.Trainer object at 0x0000026985BBCB80>}
